Generate a music video that is a latent space interpolation of stylegan3 - you define text prompts, for each prompt we generate an image, and then interpolate between them in the rythm of the music.

By [nielsrolf](github.com/nielsrolf) inspired by [mikaelalafriz's](https://github.com/mikaelalafriz) [lucid-sonic-dreams](https://github.com/mikaelalafriz/lucid-sonic-dreams), using [StyleGAN-3](https://github.com/NVlabs/stylegan3) by NVidia and OpenAIs [CLIP](https://github.com/openai/CLIP), with code for finding the latent points that tell the story taken from nshepperd and Katherine Crowson.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Music upload
audio_file = "" #@param {type: "string"}
# alternative link to youtube or soundcloud
youtube_dl_link = "https://www.youtube.com/watch?v=QehH-JWobEQ" #@param {type: "string"}

# Topic that the video starts with
# story = "Four guys with long hair and guitars->Four doors->A shaman in a desert->Psychedelic cacti->The woodstock festival->Happy hippies->Jim Morrison->A fake shaman that is for some reason once again busy making sushi" #@param {type: "string"}
story = "Four guys with long hair and guitars in a desert->Four doors->Psychedelic cacti in a desert" #@param {type: "string"}

text_prompt_bass = 'Mysterious and deep, violet' #@param {type: "string"}
text_prompt_treble = "Dreamy and warm, full of love - trending on ArtStation" #@param {type: "string"}
text_prompt_mids = "" #@param {type: "string"}

style_suffix = "Painted by Rene Magritte|trending on ArtStation" #@param {type: "string"}


# How much of a puslating effect the bass creates. A bass sound moves to the text_prompt_bass and moves back once it is released
bass_pulse_impact = 0.06 #@param {type: "number"}
# How much of a puslating effect the mids create. A mid sound moves to the text_prompt_mids and moves back once it is released
mids_pulse_impact = 0 #@param {type: "number"}
# How much of a puslating effect a high pitched sound creates. A treble sound moves to the text_prompt_trebles and moves back once it is released
trebles_pulse_impact = 0 #@param {type: "number"}

# How much the bass pushes the story forward
bass_story_speed = 1 #@param {type: "number"}
# How much the mids push the story forward
mids_story_speed = 1 #@param {type: "number"}
# How much the trebles push the story forward
trebles_story_speed = 0 #@param {type: "number"}


# It can take quite long to generate a 6min video, use these inputs to make the video shorter
start_second =  26#@param {type: "number"}
end_second = 56 #@param {type: "number"}




# Speed at which to try approximating the text. Too fast seems to give strange results. Maximum is 100.
speed = 20  #@param {type: "number"}

# Change the seed to generate variations of the same prompt 
seed = 244 #@param {type: "number"}

# Model type to use.
image_model = "imagenet"  #@param ['imagenet', 'ruDALLE']



output_path = "/content/output"

steps = 150

#@markdown ---




# Preprocess inputs

In [ ]:
story_prompts = story.split("->")
story_prompts = [f"{prompt}|{style_suffix}" if prompt!="" else "" for prompt in story_prompts]
story_prompts

In [ ]:
smoothing = (100.0-speed)/100.0

summed_speed = bass_story_speed + mids_story_speed + trebles_story_speed
bass_story_speed /= summed_speed
mids_story_speed /= summed_speed
trebles_story_speed /= summed_speed

In [ ]:
if youtube_dl_link.startswith("http"):
  print(f"Downloading from {youtube_dl_link}...")
  !pip install -q youtube-dl
  !youtube-dl --rm-cache-dir
  !youtube-dl --extract-audio --audio-format wav {youtube_dl_link} --output /tmp/audio_file.wav
  audio_file = "/tmp/audio_file.wav"
  from glob import glob
  print(glob("/tmp/*.wav"))


import librosa
import librosa.display
from matplotlib import pyplot as plt
import numpy as np

def specshow(spec):
  fig, ax = plt.subplots()
  img = librosa.display.specshow(spec, x_axis='time',
                         y_axis='mel', sr=sr,
                         fmax=8000, ax=ax)
  plt.show()


audio, sr = librosa.load(audio_file)
audio = audio[start_second*sr:end_second*sr]
spec = librosa.feature.melspectrogram(y=audio, sr=sr)[:,::2]
specshow(spec)
# spec_s = np.log(1 + spec*10)
spec_s = librosa.amplitude_to_db(spec)
spec_s = spec_s - spec_s.min()
specshow(spec_s)

seconds = len(audio) / sr
frame_rate = spec_s.shape[1] / seconds

In [ ]:
def specshow(spec):
  fig, ax = plt.subplots()
  img = librosa.display.specshow(spec, x_axis='time',
                         y_axis='mel', sr=sr,
                         fmax=8000, ax=ax)
  plt.show()
  plt.figure(figsize=(15, 7))
  ld = spec.sum(0)
  plt.plot(ld)
  plt.show()

mids = spec_s[12:-35]

bass = spec_s[:12]
bass = bass - bass.mean()
bass[bass<0] = 0

treble = spec_s[-35:]
treble = treble - treble.mean() / 2
treble[treble<0] = 0

specshow(bass)
specshow(mids)
specshow(treble)


# Find the latent points

In [ ]:
#@title Licensed under the MIT License { display-mode: "form" }

# Copyright (c) 2021 nshepperd; Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
# Check GPU and CUDA
!nvidia-smi
!nvcc --version

In [ ]:
!mkdir -p $output_path

!git clone https://github.com/openai/CLIP
!git clone https://github.com/CompVis/taming-transformers
#%cd taming-transformers
#!git checkout 2908a53b88478e5812d619b6ac003dbb29b069a0
#%cd -
!pip install ftfy regex tqdm omegaconf pytorch-lightning
!pip install kornia
!pip install einops

In [ ]:
from pathlib import Path
!sudo apt install aria2

model_mapping = {
    "ruDALLE": { 
        "name": "vqgan_openimages_f16_8192.ckpt",
        "config": "https://pollinations.ai/ipfs/QmXey26KJ1S5fc5gtrXbGqdpgc3xvoQiApVYCxzE5uB9D4/vqgan.gumbelf8-sber.model.ckpt.yaml",
        "checkpoint": "https://pollinations.ai/ipfs/QmXey26KJ1S5fc5gtrXbGqdpgc3xvoQiApVYCxzE5uB9D4/vqgan.gumbelf8-sber.model.ckpt",
    },
    "imagenet": {
        "name": "vqgan_imagenet_f16_16384.ckpt",
        "config": 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1',
        "checkpoint": 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1',        
    } 
}

selected_vqgan = model_mapping[image_model]
vqgan_name = selected_vqgan["name"]

config = selected_vqgan["config"]
checkpoint = selected_vqgan["checkpoint"]

if not Path(vqgan_name).exists():
    !aria2c -x 5 --auto-file-renaming=false '{config}' -o {vqgan_name}.yaml
    !aria2c -x 5 --auto-file-renaming=false '{checkpoint}'  -o {vqgan_name}

In [ ]:
import argparse
import math
from pathlib import Path
import sys

sys.path.append('./taming-transformers')

from IPython import display
from omegaconf import OmegaConf
from PIL import Image
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm
import numpy as np

from CLIP import clip

import kornia.augmentation as K


def noise_gen(shape):
    n, c, h, w = shape
    noise = torch.zeros([n, c, 1, 1])
    for i in reversed(range(5)):
        h_cur, w_cur = h // 2**i, w // 2**i
        noise = F.interpolate(noise, (h_cur, w_cur), mode='bicubic', align_corners=False)
        noise += torch.randn([n, c, h_cur, w_cur]) / 5
    return noise


def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))


def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()


def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]


def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size

    input = input.view([n * c, 1, h, w])

    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])

    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])

    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
    

# def replace_grad(fake, real):
#     return fake.detach() - real.detach() + real


class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward

    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)


class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)

    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None

replace_grad = ReplaceGrad.apply

clamp_with_grad = ClampWithGrad.apply
# clamp_with_grad = torch.clamp

def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)


class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))

    def forward(self, input):
        
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize((self.embed).unsqueeze(0), dim=2)

        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()


def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])

def one_sided_clip_loss(input, target, labels=None, logit_scale=100):
    input_normed = F.normalize(input, dim=-1)
    target_normed = F.normalize(target, dim=-1)
    logits = input_normed @ target_normed.T * logit_scale
    if labels is None:
        labels = torch.arange(len(input), device=logits.device)
    return F.cross_entropy(logits, labels)

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow

        self.av_pool = nn.AdaptiveAvgPool2d((self.cut_size, self.cut_size))
        self.max_pool = nn.AdaptiveMaxPool2d((self.cut_size, self.cut_size))

    def set_cut_pow(self, cut_pow):
      self.cut_pow = cut_pow

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        cutouts_full = []
        
        min_size_width = min(sideX, sideY)
        lower_bound = float(self.cut_size/min_size_width)
        
        for ii in range(self.cutn):
            
            
          size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)

          offsetx = torch.randint(0, sideX - size + 1, ())
          offsety = torch.randint(0, sideY - size + 1, ())
          cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
          cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))

        
        cutouts = torch.cat(cutouts, dim=0)

        if args.use_augs:
          cutouts = augs(cutouts)

        if args.noise_fac:
          facs = cutouts.new_empty([cutouts.shape[0], 1, 1, 1]).uniform_(0, args.noise_fac)
          cutouts = cutouts + facs * torch.randn_like(cutouts)
        

        return clamp_with_grad(cutouts, 0, 1)


def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    elif config.model.target == 'taming.models.vqgan.GumbelVQ':
        model = vqgan.GumbelVQ(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model

def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

class TVLoss(nn.Module):
    def forward(self, input):
        input = F.pad(input, (0, 1, 0, 1), 'replicate')
        x_diff = input[..., :-1, 1:] - input[..., :-1, :-1]
        y_diff = input[..., 1:, :-1] - input[..., :-1, :-1]
        diff = x_diff**2 + y_diff**2 + 1e-8
        return diff.mean(dim=1).sqrt().mean()

class GaussianBlur2d(nn.Module):
    def __init__(self, sigma, window=0, mode='reflect', value=0):
        super().__init__()
        self.mode = mode
        self.value = value
        if not window:
            window = max(math.ceil((sigma * 6 + 1) / 2) * 2 - 1, 3)
        if sigma:
            kernel = torch.exp(-(torch.arange(window) - window // 2)**2 / 2 / sigma**2)
            kernel /= kernel.sum()
        else:
            kernel = torch.ones([1])
        self.register_buffer('kernel', kernel)

    def forward(self, input):
        n, c, h, w = input.shape
        input = input.view([n * c, 1, h, w])
        start_pad = (self.kernel.shape[0] - 1) // 2
        end_pad = self.kernel.shape[0] // 2
        input = F.pad(input, (start_pad, end_pad, start_pad, end_pad), self.mode, self.value)
        input = F.conv2d(input, self.kernel[None, None, None, :])
        input = F.conv2d(input, self.kernel[None, None, :, None])
        return input.view([n, c, h, w])

class EMATensor(nn.Module):
    """implmeneted by Katherine Crowson"""
    def __init__(self, tensor, decay):
        super().__init__()
        self.tensor = nn.Parameter(tensor)
        self.register_buffer('biased', torch.zeros_like(tensor))
        self.register_buffer('average', torch.zeros_like(tensor))
        self.decay = decay
        self.register_buffer('accum', torch.tensor(1.))
        self.update()
    
    @torch.no_grad()
    def update(self):
        if not self.training:
            raise RuntimeError('update() should only be called during training')

        self.accum *= self.decay
        self.biased.mul_(self.decay)
        self.biased.add_((1 - self.decay) * self.tensor)
        self.average.copy_(self.biased)
        self.average.div_(1 - self.accum)

    def forward(self):
        if self.training:
            return self.tensor
        return self.average





In [ ]:
from functools import lru_cache

step_size = 0.1 
if vqgan_name == 'vqgan_openimages_f16_8192.ckpt':
  step_size=0.005
else:
  step_size=0.1

args = argparse.Namespace(
    size=[640, 480], 
    init_image= None,
    init_weight= 0.5,

    # clip model settings
    clip_model='ViT-B/32',
    vqgan_config=f'{vqgan_name}.yaml',         
    vqgan_checkpoint=vqgan_name,
    step_size=step_size,
    
    # cutouts / crops
    cutn=32,
    cut_pow=1,
    cut_size=224,

    # display
    display_freq=5,
    seed=None,
    use_augs = True,
    noise_fac= 0.1,

    record_generation=True,

    # noise and other constraints
    use_noise = None,
    constraint_regions = False,#
    
    
    # add noise to embedding
    noise_prompt_weights = None,
    noise_prompt_seeds = [14575],#

    # mse settings
    mse_withzeros = True,
    mse_decay_rate = 50,
    mse_epoches = 5,

    # end itteration
    max_itter = 600,
)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)


# <AUGMENTATIONS>
augs = nn.Sequential(
    K.RandomHorizontalFlip(p=0.5),
    K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'), # padding_mode=2
    K.RandomPerspective(0.2,p=0.4, ),
    K.ColorJitter(hue=0.01, saturation=0.01, p=0.7),
)

def synth_gumbel(z, quantize=True, saturate=True):
    logits = model.quantize.proj(z)
    if quantize:
        one_hot = F.gumbel_softmax(logits, tau=1, hard=True, dim=1)
    else:
        one_hot = F.one_hot(logits.argmax(1), logits.shape[1]).movedim(3, 1).to(logits.dtype)
    z_q = torch.einsum('nchw,cd->ndhw', one_hot, model.quantize.embed.weight)
    return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)


def synth(z, quantize=True, saturate=True, i=args.max_itter):
    out = None
    if args.vqgan_checkpoint == 'vqgan_openimages_f16_8192.ckpt':
      out = synth_gumbel(z, quantize, saturate)
    else:
      if args.constraint_regions:
        z = replace_grad(z, z * z_mask)

      if quantize:
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)
      else:
        z_q = z

      out = clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

    if saturate:
      progress = i / args.max_itter
      saturation = max(0,min(1,(progress - 0.25) * 2))
      out = transforms.functional.adjust_saturation(out, saturation)
    
    return out





@lru_cache(None)
def get_latents(text_input, z):
  prompts=[t.strip() for t in text_input.split("|")]

  global mse_weight
  global z_orig
  

  

  mse_decay = 0
  if args.init_weight:
    mse_decay = args.init_weight / args.mse_epoches

  noise = noise_gen([1, 3, args.size[0], args.size[1]])
  image = TF.to_pil_image(noise.div(5).add(0.5).clamp(0, 1)[0])
  image.save('init3.png')

  print('Using device:', device)
  print('using prompts: ', prompts)

  tv_loss = TVLoss() 
  mse_weight = args.init_weight

  cut_size = args.cut_size
  # e_dim = model.quantize.e_dim

  if args.vqgan_checkpoint == 'vqgan_openimages_f16_8192.ckpt':
      e_dim = 256
      n_toks = model.quantize.n_embed
      z_min = model.quantize.embed.weight.min(dim=0).values[None, :, None, None]
      z_max = model.quantize.embed.weight.max(dim=0).values[None, :, None, None]
  else:
      e_dim = model.quantize.e_dim
      n_toks = model.quantize.n_e
      z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
      z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]


  make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)

  f = 2**(model.decoder.num_resolutions - 1)
  toksX, toksY = args.size[0] // f, args.size[1] // f

  if args.seed is not None:
      torch.manual_seed(args.seed)

  if z is None:
    if args.init_image:
        pil_image = Image.open(args.init_image).convert('RGB')
        pil_image = pil_image.resize((toksX * 16, toksY * 16), Image.LANCZOS)
        pil_image = TF.to_tensor(pil_image)
        if args.use_noise:
          pil_image = pil_image + args.use_noise * torch.randn_like(pil_image) 
        z, *_ = model.encode(pil_image.to(device).unsqueeze(0) * 2 - 1)

    else:
        
        one_hot = F.one_hot(torch.randint(n_toks, [toksY * toksX], device=device), n_toks).float()

        if args.vqgan_checkpoint == 'vqgan_openimages_f16_8192.ckpt':
            z = one_hot @ model.quantize.embed.weight
        else:
            z = one_hot @ model.quantize.embedding.weight
        z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)

  if args.mse_withzeros and not args.init_image:
    z_orig = torch.zeros_like(z)
  else:
    z_orig = z.clone()

  z.requires_grad = True

  opt = optim.Adam([z], lr=args.step_size, weight_decay=0.00000000)

  normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                  std=[0.26862954, 0.26130258, 0.27577711])

  pMs = []

  if args.noise_prompt_weights and args.noise_prompt_seeds:
    for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
      gen = torch.Generator().manual_seed(seed)
      embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
      pMs.append(Prompt(embed, weight).to(device))

  for prompt in prompts:
      txt, weight, stop = parse_prompt(prompt)
      embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
      pMs.append(Prompt(embed, weight, stop).to(device))

  @torch.no_grad()
  def checkin(i, losses):
      losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
      tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
      out = synth(z, True, True, i)# False)

      TF.to_pil_image(out[0].cpu()).save(f'output/{text_input}-{i}.png')   
      #display.display(display.Image('progress.png')) 


  def ascend_txt(i):
      global mse_weight

      out = synth(z, i=i)
      
      cutouts = make_cutouts(out)
      cutouts = resample(cutouts, (perceptor.visual.input_resolution, perceptor.visual.input_resolution))


      iii = perceptor.encode_image(normalize(cutouts)).float()

      result = []

      if args.init_weight:
          
          global z_orig
          
          result.append(F.mse_loss(z, z_orig) * mse_weight / 2)
          # result.append(F.mse_loss(z, z_orig) * ((1/torch.tensor((i)*2 + 1))*mse_weight) / 2)

          with torch.no_grad():
            if i > 0 and i%args.mse_decay_rate==0 and i <= args.mse_decay_rate*args.mse_epoches:

              if mse_weight - mse_decay > 0 and mse_weight - mse_decay >= mse_decay:
                mse_weight = mse_weight - mse_decay
                print(f"updated mse weight: {mse_weight}")
              else:
                mse_weight = 0
                print(f"updated mse weight: {mse_weight}")

      for prompt in pMs:
          result.append(prompt(iii))

      return result

  def train(i):
      
      opt.zero_grad()
      lossAll = ascend_txt(i)

      if i % args.display_freq == 0:
          checkin(i, lossAll)
      
      loss = sum(lossAll)

      loss.backward()
      opt.step()

  i = 0
  with tqdm() as pbar:
      while True and i != args.max_itter:

          train(i)

          if i > 0 and i%args.mse_decay_rate==0 and i <= args.mse_decay_rate * args.mse_epoches:
            
            opt = optim.Adam([z], lr=args.step_size, weight_decay=0.00000000)

          i += 1
          pbar.update()
  
  return z.detach()





In [ ]:
z = None
latent_story = []
for story_prompt in story_prompts:
  # z = get_latents(story_prompt, z)
  z = get_latents(story_prompt, None)
  latent_story.append(z)

In [ ]:
if trebles_pulse_impact > 0:
  latent_treble = get_latents(text_prompt_treble, None)
if mids_pulse_impact > 0:
  latent_mids = get_latents(text_prompt_mids, None)
if bass_pulse_impact > 0:
  latent_bass = get_latents(text_prompt_bass, None)

del perceptor

# Interpolate

In [ ]:
def slerp(val, low, high):
  with torch.no_grad():
    val = val[:,None]
    shape = low.shape
    low = low.reshape([low.shape[0], -1])
    high = high.reshape([high.shape[0], -1])

    low_ = low / torch.norm(low, dim=1, keepdim=True)
    high_ = high / torch.norm(high, dim=1, keepdim=True)
    omega = torch.arccos(torch.clip(torch.sum(low_*high_, axis=1, keepdim=True), -1, 1))
    so = torch.sin(omega)
    so = 0*low + 0*val + so # broadcast
    slerped = (1.0-val) * low + val * high
    slerped_1 = torch.sin((1.0-val)*omega) / so * low + torch.sin(val*omega) / so * high
    slerped[so!=0] = slerped_1[so!=0]
    return slerped.reshape([slerped.shape[0]] + list(shape[1:]))

In [ ]:
def get_spec_slice(spec, i):
  tsteps = spec.shape[1]
  chapters = len(story_prompts) - 1
  chapter_len = tsteps / chapters
  start = int(i * chapter_len)
  end = int((i+1) * chapter_len)
  return spec[:,start:end]


latent_chapters = []
for chapter in range(len(story_prompts)-1):
  # Interpolation
  latent_start = latent_story[chapter]
  latent_end = latent_story[chapter + 1]
  b, m, t = get_spec_slice(bass, chapter), get_spec_slice(mids, chapter), get_spec_slice(treble, chapter)
  story_speed = b.sum(0) * bass_story_speed \
              + m.sum(0) * mids_story_speed \
              + t.sum(0) * trebles_story_speed
  print(story_speed.shape)
  story_speed = story_speed / story_speed.sum()
  progress = torch.tensor(story_speed.cumsum(0)).to(device)
  chapter = slerp(progress, latent_start, latent_end)

  # # Pulse
  # if trebles_pulse_impact > 0:
  #   bright = t.sum(0)
  #   bright = (bright - bright.min()) / (bright.max() - bright.min())
  #   bright = bright * trebles_pulse_impact
  #   bright = torch.tensor(bright).to(device)
  #   latent_story = slerp(bright, latent_story, latent_treble)

  # if mids_pulse_impact > 0:
  #   middle = m.sum(0)
  #   middle = (middle - middle.min()) / (middle.max() - middle.min())
  #   middle = middle * mids_pulse_impact
  #   middle = torch.tensor(middle).to(device)
  #   latent_story = slerp(middle, latent_story, latent_mids)

  # if bass_pulse_impact > 0:
  #   deep = b.sum(0)
  #   deep = (deep - deep.min()) / (deep.max() - deep.min())
  #   deep = deep * bass_pulse_impact
  #   deep = torch.tensor(deep).to(device)
  #   chapter = slerp(deep, chapter, latent_bass)

  latent_chapters.append(chapter)

latent_story = torch.cat(latent_chapters, dim=0)
latent_story.shape

In [ ]:
latents = latent_story


In [ ]:
!mv parts old
!rm -rf parts

In [ ]:
import os
os.makedirs("parts", exist_ok=True)

torch.cuda.empty_cache()


def write_frames(frames, start_id):
  for offset in range(len(frames)):
    frame_id = start_id + offset
    TF.to_pil_image(frames[offset]).save(f'parts/output_{frame_id:08}.jpg')

batch = 10
start_frame = 0
end_frame = batch
part_id = 0
while end_frame < len(latents):
  with torch.no_grad():
    frames = synth(latents[start_frame:end_frame], quantize=True, saturate=True).cpu()
    # frames = synth(latents[start_frame:end_frame], quantize=False, saturate=True).cpu()
    write_frames(frames, start_frame)
    start_frame = end_frame
    end_frame += batch

In [ ]:

import soundfile as sf
sf.write("audio_cut.wav", audio, sr)

In [ ]:
!rm *.avi
!ffmpeg  -r {frame_rate} -i parts/%*.jpg -y -c:v libx264 vid_no_audio.mp4
!ffmpeg -y -i audio_cut.wav -i vid_no_audio.mp4 final_video.mp4

In [ ]:
!mkdir {output_path}
!cp -v final_video.mp4 {output_path}
!cp -v final_video.mp4 /content/drive/MyDrive/doors.mp4
!sleep 20

Notes for stylegan notebook:
- add torch.no_grad to slerp
- use correct parameter for mids_pulse_impact


In [ ]:
!cp -v final_video.mp4 /content/drive/MyDrive/doors.mp4